*First we are mounting drive for the google drive and then importing Python SDK for AWS  i.e boto3*

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!pwd
!ls - "/content/drive/My Drive/vibhawdrivepy"
import boto3, os

/content


***Resource is  the high level service*** *class recommended to be used. This allow you to tied particular AWS resources and pass it along, so you just use this abstraction than worry which target services is pointed to.*

In [0]:
#Using Resource
resource= boto3.resource(
    's3',
    aws_access_key_id='your access key',
    aws_secret_access_key='your secret key',
    
)


***Client is a low level object***.* for each client call .For each client call, you need to explicitly specify the targeting resources, the designated service target name must be pass long. You will lost the abstraction ability*

In [0]:
#Using Client . This we will go in details in other examples latter
resource= boto3.client(
    's3',
    aws_access_key_id='your access key',
    aws_secret_access_key='your secret key',
    
)

***Sessions***:*where initiate the connectivity to AWS services. E.g. following is default session that use the default credential profile. Because default session is limit to the profile or instance profile used, sometime you need to use the custom session to override the default session configuration (e.g. region_name, endpoint_url, etc. ) *


In [0]:
s3 = boto3.resource('s3')

my_west_session = boto3.Session(region_name = 'us-west-2')
my_east_sesison = boto3.Session(region_name = 'us-east-1')

# custom resource session must use boto3.Session to do the override
backup_s3 = my_west_session.resource('s3')
video_s3 = my_east_sesison.resource('s3')


# you have two choices of create custom client session. 
backup_s3c = my_west_session.client('s3')
video_s3c = boto3.client("s3", region_name = 'us-east-1')
                  

**List buckets along with the file name**

this we do in AWS CLI :

! aws s3 ls

! aws s3 ls demo-bucket-prudev/ --recursive

! aws s3 ls --recursive 

In [0]:
for bucket in resource.buckets.all():
    for obj in bucket.objects.all():
        print(os.path.join(obj.bucket_name, obj.key))

athenabucketvibs/affectedplatforms/
athenabucketvibs/dates/
athenabucketvibs/output/
athenabucketvibs/resolve/
athenabucketvibs/resolve/serviceone.txt
athenabucketvibs/users/
athenabucketvibs/users/user.txt
athenoutputquery/output/
demo-bucket-prudev/newbook.csv
demo-bucket-prudev/newbook.xls
redshifts3bucketvibs/affectedplatform.txt
redshifts3bucketvibs/affectplatform.txt
redshifts3bucketvibs/assgrp.txt
redshifts3bucketvibs/category.txt
redshifts3bucketvibs/date.txt
redshifts3bucketvibs/date2.txt
redshifts3bucketvibs/date3.txt
redshifts3bucketvibs/priorty.txt
redshifts3bucketvibs/sc_request_tousefinal.txt
redshifts3bucketvibs/sc_request_tousevibs.txt
redshifts3bucketvibs/serreq.txt
redshifts3bucketvibs/state.txt
redshifts3bucketvibs/user.txt


**Creating a bucket**

*Bucket names need to be globally unique, meaning that no two buckets can have the same name, not even when they are owned by different users.  P.S : - use uuid4 for this (we will see that in our examples later down)*

! aws s3 mb s3://demo-bucket-prudev

In [0]:
resource.create_bucket(Bucket='demo-bucket-prudev')

s3.Bucket(name='demo-bucket-prudev')

 **Upload file
! aws s3 cp test.txt s3://demo-bucket-prudev/**

In [0]:
resource.meta.client.upload_file('/content/drive/My Drive/vibhawdrivepy/newbook.csv', 'demo-bucket-prudev', 'newbook.csv')

**Delete object **  ! aws s3 rm s3://demo-bucket-prudev/newbook.xls

In [0]:
resource.meta.client.delete_object(Bucket="demo-bucket-prudev", Key="newbook.xls")

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Mon, 28 Jan 2019 19:02:13 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'ZqxhLek9GLHmqlb0YEWuNPOcOlDTj/Hxj47OeitkND+buEfhLPVxck5yfb7S2tqeeVlHCVeX3GE=',
   'x-amz-request-id': 'DF14931A3F475437'},
  'HTTPStatusCode': 204,
  'HostId': 'ZqxhLek9GLHmqlb0YEWuNPOcOlDTj/Hxj47OeitkND+buEfhLPVxck5yfb7S2tqeeVlHCVeX3GE=',
  'RequestId': 'DF14931A3F475437',
  'RetryAttempts': 0}}

In [0]:
obj = resource.Object("demo-bucket-prudev", "newbook.xls")
obj.delete()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Mon, 28 Jan 2019 19:04:06 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'AtnpB7Ps7LlFdKMb4EdU/bhaaUfQt/6CTDGEe7S3eTo/JqyoheGhS0murc+duX0Jf1cZulfEFRU=',
   'x-amz-request-id': 'D4D628B877179608'},
  'HTTPStatusCode': 204,
  'HostId': 'AtnpB7Ps7LlFdKMb4EdU/bhaaUfQt/6CTDGEe7S3eTo/JqyoheGhS0murc+duX0Jf1cZulfEFRU=',
  'RequestId': 'D4D628B877179608',
  'RetryAttempts': 0}}

In [0]:
resource.meta.client.upload_file('/content/drive/My Drive/vibhawdrivepy/newbook.xls', 'demo-bucket-prudev', 'newbook1.xls')

In [0]:
resource.Object('demo-bucket-prudev','moved2.txt').copy_from(CopySource='demo-bucket-prudev/newbook1.xls')

{'CopyObjectResult': {'ETag': '"d8afd2a6f971397be7e7ca0a6648e7b7"',
  'LastModified': datetime.datetime(2019, 1, 28, 19, 5, 58, tzinfo=tzlocal())},
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '234',
   'content-type': 'application/xml',
   'date': 'Mon, 28 Jan 2019 19:05:58 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'OqrkpNzu9nKfb4vDO8g2TTta9ZSLr3cMyzl0bP/00s/8Cd0hA5JjzfBEzS2PbC03eF7E17a7AQk=',
   'x-amz-request-id': '8060BEC7A64115C9'},
  'HTTPStatusCode': 200,
  'HostId': 'OqrkpNzu9nKfb4vDO8g2TTta9ZSLr3cMyzl0bP/00s/8Cd0hA5JjzfBEzS2PbC03eF7E17a7AQk=',
  'RequestId': '8060BEC7A64115C9',
  'RetryAttempts': 0}}

In [0]:
for bucket in resource.buckets.all():
    print(bucket.name)

athenabucketvibs
athenoutputquery
demo-bucket-prudev
firstpythonbucketd59eafc5-2538-4f61-96df-9cd465895435
redshifts3bucketvibs


In [0]:
for obj in resource.Bucket(name='demo-bucket-prudev').objects.all():
    print(os.path.join(obj.bucket_name, obj.key))
    
    

demo-bucket-prudev/newbook.xls


In [0]:
import re
objs = [os.path.join(obj.bucket_name, obj.key) 
        for obj in resource.Bucket(name='demo-bucket-prudev').objects.all() 
        if re.match(".*xls",obj.key)]
print("\n".join(objs))



demo-bucket-prudev/newbook.xls


In [0]:
bucket = resource.Bucket('demo-bucket-prudev')

# empty the bucket
for key in bucket.objects.all():
    key.delete()
    
# then delete bucket.delete()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Mon, 28 Jan 2019 19:11:38 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'nhjKVe+E5xctlfK4L8F/VTTz30duPlam4n+73YdtvEC5r6EHJ45HKXO0WXEshts26JALj7RlPi8=',
   'x-amz-request-id': '886A8072CF110D56'},
  'HTTPStatusCode': 204,
  'HostId': 'nhjKVe+E5xctlfK4L8F/VTTz30duPlam4n+73YdtvEC5r6EHJ45HKXO0WXEshts26JALj7RlPi8=',
  'RequestId': '886A8072CF110D56',
  'RetryAttempts': 0}}

In [0]:
import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

In [0]:
first_bucket_name = create_bucket_name('firstpythonbucket')
first_bucket_name

'firstpythonbucketee234a75-7e1a-4663-bcec-ee0168fb6c9c'

In [0]:
s3 = boto3.client(
    's3',
    aws_access_key_id='your access key',
    aws_secret_access_key='your secret key',
)
obj =s3.get_object(Bucket='demo-bucket-prudev', Key='newbook.csv')
#j= obj['Body'].read().decode('ISO-8859-1')
j= obj['Body'].read().decode('utf-8')
print(j)

﻿master_plan_id,rule_type,rule_seq,rule_id,rule_txt,date_upd,vru_rule,vru_rule_type
100001,RS,102,111,hello,10/11/2018,1,1
100002,RS,103,111,hello,10/12/2018,1,1
100003,RS,104,111,hello,10/13/2018,1,1
100004,RS,105,111,hello,10/14/2018,1,1
100005,RS,106,111,hello,10/15/2018,1,1
100006,RS,107,111,hello,10/16/2018,1,1
100007,RS,108,111,hello,10/17/2018,1,1



In [0]:
s3 = boto3.resource(
    's3',
   aws_access_key_id='your access key',
    aws_secret_access_key='your secret key',
    
)
obj =s3.meta.client.get_object(Bucket='demo-bucket-prudev', Key='newbook.csv')
j= obj['Body'].read().decode('utf-8')
print(j)

﻿master_plan_id,rule_type,rule_seq,rule_id,rule_txt,date_upd,vru_rule,vru_rule_type
100001,RS,102,111,hello,10/11/2018,1,1
100002,RS,103,111,hello,10/12/2018,1,1
100003,RS,104,111,hello,10/13/2018,1,1
100004,RS,105,111,hello,10/14/2018,1,1
100005,RS,106,111,hello,10/15/2018,1,1
100006,RS,107,111,hello,10/16/2018,1,1
100007,RS,108,111,hello,10/17/2018,1,1



**Queues in Amazon SQS**

The code uses the AWS SDK for Python to use queues using these methods of the AWS.SQS client class:

    list_queues.
    create_queue.
    get_queue_url.
    delete_queue.

In [0]:
#List Your Queues
import boto3

# Create SQS client
sqs = boto3.client('sqs',aws_access_key_id='your key',
    aws_secret_access_key='your secret key',region_name= 'us-east-1')

# List SQS queues
response = sqs.list_queues()

print(response)
print(response['QueueUrls'])

{'QueueUrls': ['https://queue.amazonaws.com/716349339227/SQS_QUEUE_NAME'], 'ResponseMetadata': {'RequestId': 'de28cce8-0e26-5a69-9793-982266990d49', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'de28cce8-0e26-5a69-9793-982266990d49', 'date': 'Tue, 29 Jan 2019 14:43:11 GMT', 'content-type': 'text/xml', 'content-length': '322'}, 'RetryAttempts': 0}}
['https://queue.amazonaws.com/716349339227/SQS_QUEUE_NAME']


In [0]:
# Create a SQS queue
response = sqs.create_queue(
    QueueName='SQS_QUEUE_NAME',
    Attributes={
        'DelaySeconds': '60',
        'MessageRetentionPeriod': '86400'
    }
)

print(response['QueueUrl'])

https://queue.amazonaws.com/716349339227/SQS_QUEUE_NAME


In [0]:
# Get URL for SQS queue
response = sqs.get_queue_url(QueueName='SQS_QUEUE_NAME')

print(response['QueueUrl'])

https://queue.amazonaws.com/716349339227/SQS_QUEUE_NAME


In [0]:
#Send a Message to a Queue
#queue_url = sqs.get_queue_url(QueueName='SQS_QUEUE_NAME')
import boto3

# Create SQS client
sqs = boto3.client('sqs',aws_access_key_id='your key',
    aws_secret_access_key='your secrect key',region_name= 'us-east-1')


queues = sqs.list_queues(QueueNamePrefix='SQS_QUEUE_NAME')
#print(queues)
test_queue_url = queues['QueueUrls'][0]
#test_queue_url
enqueue_response = sqs.send_message(QueueUrl=test_queue_url, MessageBody='This is test message #')
print('Message ID : ',enqueue_response['MessageId'])

enqueue_response = sqs.send_message(
    QueueUrl=test_queue_url,
    DelaySeconds=10,
    MessageAttributes={
        'Title': {
            'DataType': 'String',
            'StringValue': 'Training'
        },
        'Details': {
            'DataType': 'String',
            'StringValue': 'On Python SDK Boto AWS'
        },
        'WeeksOn': {
            'DataType': 'Number',
            'StringValue': '2'
        }
    },
    MessageBody=(
        'Information about current Current Training on Python on going  '
        'week of 01/29/2019.'
    )
)

print(enqueue_response['MessageId'])



Message ID :  e1ebdf37-4b6d-4cb6-ad9c-b5b57554d907
977891e8-da71-4ba3-9be5-ba91e25d7026


In [0]:
sqs = boto3.client('sqs',aws_access_key_id='your key',
    aws_secret_access_key='your secret key',region_name= 'us-east-1')

queues = sqs.list_queues(QueueNamePrefix='SQS_QUEUE_NAME')
#print(queues)
test_queue_url = queues['QueueUrls'][0]

# Receive message from SQS queue
response = sqs.receive_message(
    QueueUrl=test_queue_url,
    AttributeNames=[
        'SentTimestamp'
    ],
    MaxNumberOfMessages=1,
    MessageAttributeNames=[
        'All'
    ],
    VisibilityTimeout=0,
    WaitTimeSeconds=0
)

message = response['Messages'][0]
receipt_handle = message['ReceiptHandle']

# Delete received message from queue
sqs.delete_message(
    QueueUrl=test_queue_url,
    ReceiptHandle=receipt_handle
)
print('Received and deleted message: %s' % message)

Received and deleted message: {'MessageId': '977891e8-da71-4ba3-9be5-ba91e25d7026', 'ReceiptHandle': 'AQEB0CYSNUP85uAnLMDcq3IS88wOBrQePRXHm5jfzpcGIfpPEZo1m4oxff/201+2KoETYHUIcRcQ2iFThD0wo3dfW+Za9xFlsqOdvsLyeYcatzhXUpQfgxtv1WEjUV1G9d/sltmm2njK2PR+l+doMo9a8sgUfhjft7eJR3OZLsiJrkcLZGgFcSrLC53bNNiFkwkCBsfjp9eGGXLmsPEqgkiZCKLsQ0WL/HB/DH0CRg6WW8bhL8BaZ7rL0lATIA3AKP6TnfqkRu2XUPhL3eN2Bh0yM87a02JNiMpscJgj9ZVXFv1s55NYUKteZFdbMh5OtWMaNWet86k7xKxyWm4NUweEKXbs95IzYz5OAwF7To2by+oTqge/HiiX45znEAgiFvxtpg7mKcmldhEjxbsaeaRxCg==', 'MD5OfBody': 'd8ef0c71072305cfbf2a8d6e9e77ae48', 'Body': 'Information about current Current Training on Python on going  week of 01/29/2019.', 'Attributes': {'SentTimestamp': '1548773891284'}, 'MD5OfMessageAttributes': 'c4a612a6b0f76f797e2a2f0ee6395891', 'MessageAttributes': {'Details': {'StringValue': 'On Python SDK Boto AWS', 'DataType': 'String'}, 'Title': {'StringValue': 'Training', 'DataType': 'String'}, 'WeeksOn': {'StringValue': '2', 'DataType': 'Number'}}}


**AWS SDK for Python to manage the instances by using these methods of the EC2 client class:**

    describe_instances.
    monitor_instances.
    unmonitor_instances.
    start_instances.
    stop_instances.
    reboot_instances.

Try the same with your AWS account .